# 텍스트 지뢰찾기

## 목적
* ...
* DFS 알고리즘으로 지뢰찾기 일부 기능을 구현한다.  
클릭한 곳이 빈 곳일(숫자 또는 지뢰가 아닌 부분) 경우 그 주변에 연결된 빈 공간을 찾는다.(빈 공간은 숫자로 둘러쌓여 있음)  
숫자로 둘러 쌓인 빈공간들을 구분 한다.
* GUI가 아닌 Text로 지뢰찾기 게임을 구현 한다.
* ...

In [10]:
from random import shuffle
from itertools import cycle

rows       = 16 
cols       = 16
mines      = 40
minePlate  = []
spacePlate = {}
display    = []
rect       = ((-1, -1), (-1, 0), (-1, 1),
              (0, -1),            (0, 1),
              (1, -1),  (1, 0),   (1, 1)
             )

words      = {
    'realMine'     : '*',
    'markMine'     : 'M',
    'space'        : 'O',
    'openSpace'    : ' ',
    'numbers'      : '0123456789',
    'groups'       : 'ABCDEFGHIJKLMNOPQRSTUVWXYZ',
    'markQuestion' : '?',
    'initialChar'  : '#',
    'wrong'        : 'X'
}



def showDispay(target):
    '''상태를 화면에 표시한다'''
    colCodi = cycle('1234567890')
    rowCodi = cycle('1234567890')
    
    print('  ', end = '')
    for _ in range(cols):
        print(next(colCodi), end = '   ')    
    print()

    
    for x in target:
        print(next(rowCodi), ' | '.join(x))

def countMines(row, col):
    '''주변에 있는 지뢰 개수를 리턴한다.'''
    count = 0
    
    for x, y in rect:
        x += row 
        y += col
        
        if isValidCodi(x, y):
            if minePlate[x][y] == words['realMine']:
                count += 1
    
    return count

def commandClick(x, y):
    '''좌료를 클릭시 minePlate의 상태에 따라 display 상태를 변경한다'''
    # x -= 1
    # y -= 1
    status = minePlate[x][y]
    
    if status == '*':
        display[x][y] = 'X'
        return 'fail'
    elif '0' <= status <= '8':
        display[x][y] = status
    else:
        _openSpace(status, spacePlate[status])
        
def commandDoubleClidk(x, y):
    x      -= 1
    y      -= 1
    status  = display[x][y]
    
    if hasQuestion(x, y):
        return
    
    if status == words['space'] :
        return commandClick(x, y)
    
    realMines   = getMinesInfo(minePlate, x, y, words['realMine'])
    markedMines = getMinesInfo(display, x, y, words['markMine'])
    unmarked    = getMinesInfo(display, x, y, words['space'])
    
    realMines.sort()
    markedMines.sort()
    
    if len(markedMines) != 0:
        if realMines == markedMines:    
            for codi in unmarked:
                commandClick(x + codi[0], y + codi[1])
        else:
            if isValidMarkMines(realMines, markedMines):
                return 'fail'
            else:
                return
    else:
        return
            
def isValidMarkMines(real, marked):
    for ele in real:
        if ele in marked:
            marked.remove(ele)

    if marked:
        return True
    else:
        return False
    

def hasQuestion(x, y):
    return len(getMinesInfo(display, x, y, words['markQuestion'])) != 0
    
def getMinesInfo(plate, x, y, findChar):
    return list(filter(lambda c:(0 <= c[0] + x < rows and 0 <= c[1] + y < cols) and plate[c[0] + x][c[1] + y] == findChar, rect))    

def _openSpace(mark, area):
    '''연결된 빈공간을 display에 상태를 변경한다.'''
    for x, y in area:
        temp = minePlate[x][y]
        
        display[x][y] = words['openSpace'] if temp == mark else temp
        
def dfs(row, col):
    '''지뢰가 없는 영역 찾기'''
    rect = ((-1, 0), (0, -1), (0, 1), (1, 0))
    
    if minePlate[row][col] == words['initialChar']:
        minePlate[row][col] = tempGroup
        tempSpace.append((row, col))
        
        for x, y in rect:
            x += row 
            y += col

            if isValidCodi(x, y):
                dfs(x, y)
        
    elif  minePlate[row][col] in words['numbers']:
        tempSpace.append((row, col))  
        
def isValidCodi(x, y):
    return 0 <= x < rows and 0 <= y < cols

def showMines():
    for x in range(rows):
        for y in range(cols):
            if display[x][y] == words['space'] and minePlate[x][y] == words['realMine']:
                display[x][y] = words['realMine']
            
            elif display[x][y] == words['markMine'] and minePlate[x][y] != words['realMine']:
                display[x][y] = words['wrong']
    
    showDispay(display)
    
def markMine(x, y):
    x -= 1
    y -= 1
    
    if display[x][y] in [words['space'], words['markQuestion']]:
        display[x][y] = words['markMine']

def unmarkMine(x,  u):
    x -= 1
    y -= 1
    
    if display[x][y] in [words['markMine'], words['markQuestion']]:
        display[x][y] = words['space']

def init():
    # 지뢰 초기화 하기
    temp = [words['initialChar']] * (rows * cols)

    for x in range(mines):
        temp[x] = words['realMine']

    shuffle(temp)

    for x in range(rows):
        minePlate.append(temp[x * cols:(x + 1) * cols])
        display.append([words['space']] * cols)

    # 주위에 지뢰 개수 구하기
    for row in range(rows):
        for col in range(cols):
            if minePlate[row][col] == words['initialChar']:
                count = countMines(row, col)
                minePlate[row][col] = str(count) if count != 0 else words['initialChar']

    # 숫자로 둘러 쌓이 빈 공간들을 구분한다.
    mark = iter(words['groups'])
    for row in range(rows):
        for col in range(cols):
            if minePlate[row][col] == words['initialChar']:
                tempSpace = []
                tempGroup = next(mark)
                dfs(row, col)
                spacePlate[tempGroup] = tempSpace

# 게임 시작
def start():
    while True:
        command = list(input('x y | D x y| M x y | U x y | V | S | Q ? ').split())

        commandLength = len(command)

        # V S Q 명령
        if commandLength == 1:
            if command[0].lower() == 'v':   # 게임 진행 상태를 표시
                    showDispay(display)
            elif command[0].lower() == 'q': # 게임 종료
                    break
            elif command[0].lower() == 's': # 지뢰 및 지뢰 갯수 빈공간 영역 표시
                    showDispay(minePlate)
        # D M U 명령            
        elif commandLength == 3:
            if command[0].lower() == 'd':   # 클릭
                commandDoubleClidk(int(command[1]), int(command[2]))
            elif command[0].lower() == 'm': # 지뢰 표시
                markMine(int(command[1]), int(command[2]))
            elif command[0].lower() == 'u': # 지뢰 표시 지우기 
                unmarkMine(int(command[1]), int(command[2]))
            elif command[0].lower() == '?': # 지뢰 표시 지우기 
                unmarkMine(int(command[1]), int(command[2]))
        elif commandLength == 2:
            result = commandDoubleClidk(int(command[0]), int(command[1]))

            if result == 'fail':
                print('지뢰가 폭발했어요. ㅠㅠ')
                showMines()
                break

init()
start()

x y | D x y| M x y | U x y | V | S | Q ? s
  1   2   3   4   5   6   7   8   9   0   1   2   3   4   5   6   
1 K | 1 | 1 | 2 | 3 | * | 3 | * | 1 | K | 1 | 1 | 1 | K | K | K
2 K | 1 | * | 2 | * | * | 3 | 1 | 1 | K | 2 | * | 3 | 1 | 2 | 1
3 K | 1 | 1 | 2 | 2 | 2 | 1 | K | 1 | 1 | 3 | * | 3 | * | 4 | *
4 K | K | K | K | K | K | 1 | 1 | 2 | * | 2 | 1 | 2 | 2 | * | *
5 K | K | K | K | K | K | 1 | * | 3 | 2 | 1 | 1 | 1 | 2 | 2 | 2
6 K | K | 1 | 1 | 1 | K | 1 | 2 | * | 1 | K | 1 | * | 1 | K | K
7 K | K | 1 | * | 2 | 2 | 2 | 3 | 2 | 3 | 1 | 3 | 2 | 3 | 1 | 1
8 K | 1 | 2 | 2 | 2 | * | * | 2 | * | 2 | * | 2 | * | 2 | * | 1
9 1 | 2 | * | 1 | 1 | 2 | 2 | 2 | 1 | 2 | 1 | 2 | 1 | 3 | 2 | 2
0 * | 2 | 1 | 1 | K | K | K | K | K | 1 | 1 | 1 | K | 1 | * | 2
1 1 | 1 | K | K | K | 1 | 1 | 1 | K | 1 | * | 2 | 1 | 2 | 2 | *
2 K | K | K | K | K | 1 | * | 2 | 2 | 2 | 2 | 2 | * | 2 | 3 | 2
3 K | K | K | K | K | 2 | 3 | * | 2 | * | 2 | 3 | 3 | * | 2 | *
4 1 | 1 | 1 | K | K | 1 | * | 2 | 2 | 2 | * | 2 | * | 3 | 

In [7]:
a = [1, 3, 2]
b = [1, 2, 3]

a.sort()
b.sort()

a == b

True

In [14]:
from random import shuffle
from itertools import cycle

class Minesweeper:
    def __init__(self, rows = 16, cols = 16, mineCount = 40):
        self.rows       = rows 
        self.cols       = cols
        self.mines      = mineCount
        self.minePlate  = []
        self.spacePlate = {}
        self.display    = []
        self.rect       = ((-1, -1), (-1, 0), (-1, 1),
                      (0, -1),            (0, 1),
                      (1, -1),  (1, 0),   (1, 1)
                     )

        self.words      = {
            'realMine'     : '*',
            'markMine'     : 'M',
            'space'        : 'O',
            'openSpace'    : ' ',
            'numbers'      : '0123456789',
            'groups'       : 'ABCDEFGHIJKLMNOPQRSTUVWXYZ',
            'markQuestion' : '?',
            'initialChar'  : '#',
            'wrong'        : 'X'
        }

    def showDispay(self, target):
        '''상태를 화면에 표시한다'''
        colCodi = cycle('1234567890')
        rowCodi = cycle('1234567890')

        print('  ', end = '')
        for _ in range(self.cols):
            print(next(colCodi), end = '   ')    
        print()


        for x in target:
            print(next(rowCodi), ' | '.join(x))

    def countMines(self, row, col):
        '''주변에 있는 지뢰 개수를 리턴한다.'''
        count = 0

        for x, y in self.rect:
            x += row 
            y += col

            if self.isValidCodi(x, y):
                if self.minePlate[x][y] == self.words['realMine']:
                    count += 1

        return count

    def commandClick(self, x, y):
        '''좌료를 클릭시 minePlate의 상태에 따라 display 상태를 변경한다'''
        
        status = self.minePlate[x][y]

        if status == self.words['realMine']:
            self.display[x][y] = 'X'
            return 'fail'
        elif '0' <= status <= '8':
            self.display[x][y] = status
        else:
            self._openSpace(status, self.spacePlate[status])

    def commandDoubleClidk(self, x, y):
        x      -= 1
        y      -= 1
        status  = self.display[x][y]

        if self.hasQuestion(x, y):
            return

        if status == self.words['space'] :
            return self.commandClick(x, y)

        realMines   = self.getMinesInfo(self.minePlate, x, y, self.words['realMine'])
        markedMines = self.getMinesInfo(self.display, x, y, self.words['markMine'])
        unmarked    = self.getMinesInfo(self.display, x, y, self.words['space'])

        realMines.sort()
        markedMines.sort()

        if len(markedMines) != 0:
            if realMines == markedMines:    
                for codi in unmarked:
                    self.commandClick(x + codi[0], y + codi[1])
            else:
                if self.isValidMarkMines(realMines, markedMines):
                    return 'fail'
                else:
                    return
        else:
            return

    def isValidMarkMines(self, real, marked):
        for ele in real:
            if ele in marked:
                marked.remove(ele)

        if marked:
            return True
        else:
            return False


    def hasQuestion(self, x, y):
        return len(self.getMinesInfo(self.display, x, y, self.words['markQuestion'])) != 0

    def getMinesInfo(self, plate, x, y, findChar):
        return list(filter(lambda c:(0 <= c[0] + x < self.rows and 0 <= c[1] + y < self.cols) and plate[c[0] + x][c[1] + y] == findChar, self.rect))    

    def _openSpace(self, mark, area):
        '''연결된 빈공간을 display에 상태를 변경한다.'''
        for x, y in area:
            temp = self.minePlate[x][y]

            self.display[x][y] = self.words['openSpace'] if temp == mark else temp

    def dfs(self, row, col):
        '''지뢰가 없는 영역 찾기'''
        rect = ((-1, 0), (0, -1), (0, 1), (1, 0))

        if self.minePlate[row][col] == self.words['initialChar']:
            self.minePlate[row][col] = self.tempGroup
            self.tempSpace.append((row, col))

            for x, y in rect:
                x += row 
                y += col

                if self.isValidCodi(x, y):
                    self.dfs(x, y)

        elif  self.minePlate[row][col] in self.words['numbers']:
            self.tempSpace.append((row, col))  

    def isValidCodi(self, x, y):
        return 0 <= x < self.rows and 0 <= y < self.cols

    def showMines(self):
        for x in range(self.rows):
            for y in range(self.cols):
                if self.display[x][y] == self.words['space'] and self.minePlate[x][y] == self.words['realMine']:
                    self.display[x][y] = self.words['realMine']

                elif self.display[x][y] == self.words['markMine'] and self.minePlate[x][y] != self.words['realMine']:
                    self.display[x][y] = self.words['wrong']

        self.showDispay(self.display)

    def markMine(self, x, y):
        x -= 1
        y -= 1

        if self.display[x][y] in [self.words['space'], self.words['markQuestion']]:
            self.display[x][y] = self.words['markMine']

    def unmarkMine(self, x,  u):
        x -= 1
        y -= 1

        if self.display[x][y] in [self.words['markMine'], self.words['markQuestion']]:
            self.display[x][y] = self.words['space']

    def init(self):
        # 지뢰 초기화 하기
        temp = [self.words['initialChar']] * (self.rows * self.cols)

        for x in range(self.mines):
            temp[x] = self.words['realMine']

        shuffle(temp)

        for x in range(self.rows):
            self.minePlate.append(temp[x * self.cols:(x + 1) * self.cols])
            self.display.append([self.words['space']] * self.cols)

        # 주위에 지뢰 개수 구하기
        for row in range(self.rows):
            for col in range(self.cols):
                if self.minePlate[row][col] == self.words['initialChar']:
                    count = self.countMines(row, col)
                    self.minePlate[row][col] = str(count) if count != 0 else self.words['initialChar']

        # 숫자로 둘러 쌓이 빈 공간들을 구분한다.
        mark = iter(self.words['groups'])
        for row in range(self.rows):
            for col in range(self.cols):
                if self.minePlate[row][col] == self.words['initialChar']:
                    self.tempSpace = []
                    self.tempGroup = next(mark)
                    self.dfs(row, col)
                    self.spacePlate[self.tempGroup] = self.tempSpace

    # 게임 시작
    def start(self):
        self.init()
        while True:
            command = list(input('x y | D x y| M x y | U x y | V | S | Q ? ').split())

            commandLength = len(command)

            # V S Q 명령
            if commandLength == 1:
                if command[0].lower() == 'v':   # 게임 진행 상태를 표시
                        self.showDispay(self.display)
                elif command[0].lower() == 'q': # 게임 종료
                        break
                elif command[0].lower() == 's': # 지뢰 및 지뢰 갯수 빈공간 영역 표시
                        self.showDispay(self.minePlate)
            # D M U 명령            
            elif commandLength == 3:
                if command[0].lower() == 'd':   # 클릭
                    self.commandDoubleClidk(int(command[1]), int(command[2]))
                elif command[0].lower() == 'm': # 지뢰 표시
                    self.markMine(int(command[1]), int(command[2]))
                elif command[0].lower() == 'u': # 지뢰 표시 지우기 
                    self.unmarkMine(int(command[1]), int(command[2]))
                elif command[0].lower() == '?': # 지뢰 표시 지우기 
                    self.unmarkMine(int(command[1]), int(command[2]))
            elif commandLength == 2:
                result = self.commandDoubleClidk(int(command[0]), int(command[1]))

                if result == 'fail':
                    print('지뢰가 폭발했어요. ㅠㅠ')
                    self.showMines()
                    break

In [15]:
m = Minesweeper()

m.start()

x y | D x y| M x y | U x y | V | S | Q ? v
  1   2   3   4   5   6   7   8   9   0   1   2   3   4   5   6   
1 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
2 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
3 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
4 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
5 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
6 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
7 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
8 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
9 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
0 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
1 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
2 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
3 O | O | O | O | O | O | O | O | O | O | O | O | O | O | O | O
4 O | O | O | O | O | O | O | O | O | O | O | O | O | O | 

In [50]:
class abc:
    def __init__(self):
        pass
    
    def __init__(self, a = 6):
        pass

In [51]:
abc()